In [1]:
from MotionGPT import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # device

In [3]:
loader = FBXDataLoader("Temp/", has_translate=True)
loader.LoadData()

FBXDataLoader loading data...



Hip Spine distance as scale:  3.968148177595601


In [4]:
print(len(loader.raw_data),len(loader.raw_data[0]),len(loader.raw_data[0][0]),len(loader.raw_data[0][0][0]))

4 110 65 6


In [5]:
loader.translate_scale

19.840740887978004

In [6]:
loader.PrepareTrainingData(training_joint_index=[i for i in range(len(G_MIXAMO_JOINTS))])

In [7]:
print(len(loader.all_data),len(loader.all_data[0]),len(loader.all_data[0][0]))
print(len(loader.all_data),len(loader.all_data[-1]),len(loader.all_data[-1][0]))

48 10 198
48 9 198


In [8]:
print(len(loader.train_data),len(loader.train_data[0]),len(loader.train_data[0][1]))

38 10 198


In [9]:
from SeqTransformer import *

input_dim = 198

model = build_seq_vae(input_dim, 2)

In [10]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

6266118


In [11]:
model.load_state_dict(torch.load("runs/2020_12_14_16_33_16_nr.pth"))

ModuleAttributeError: 'VRNN' object has no attribute 'copy'

In [ ]:
#build model for training
if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
batch_x, batch_mask = next(iter(loader.next_batch(batch_size=4)))

In [ ]:
batch_x = batch_x.to(device)
batch_mask = batch_mask.to(device)

batch_x_decoder_input = batch_x[:,1:, :] # Batch x (Length - 1) x Dim
batch_x_encoder_input = batch_x[:,:-1, :] # Batch x (Length - 1) x Dim

batch_mask_encoder_input = batch_mask[:, 1:] # Batch x (Length - 1)

batch_output, gaussian_params = model(batch_x_encoder_input, batch_x_decoder_input)
